In [1]:
"""
Imports
"""
import sys
print(f"Python:\t\t{sys.version}")
import numpy as np
print(f"Numpy:\t\t{np.__version__}")
import tensorflow as tf
print(f"Tensorflow:\t{tf.__version__}")
import keras
print(f"Keras:\t\t{keras.__version__}")

import os
import matplotlib.pyplot as plt
from keras import Sequential
from keras import layers

Python:		3.8.20 (default, Oct  3 2024, 15:19:54) [MSC v.1929 64 bit (AMD64)]
Numpy:		1.24.3
Tensorflow:	2.10.0
Keras:		2.10.0


In [2]:
"""
Options
"""
model_name = "DemooNet_8c_3d_60e_32b"

image_width = 128
image_height = 128
channels_dim = 3
classes_dim = 6
batch_size = 32

validation_split = 0.2
base_learning_rate = 0.001
epochs_total = 60

In [3]:
"""
Pathing
"""
SPECTROGRAMS_DIR = "../data/spectrograms/combined"
TF_MODELS_DIR = "../data/models_tf"
TFLITE_MODELS_DIR = "../data/models_tflite"

In [ ]:
"""
Configure GPU
"""
gpus = tf.config.list_physical_devices("GPU")
if gpus:
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices("GPU")
    print(f"{len(gpus)} physical GPUs\n{len(logical_gpus)} logical GPUs")
  except RuntimeError as e:
    print(f"Error: {e}")

In [4]:
"""
Functions
"""
def resize_or_crop(image, label, target_height, target_width):
    shape = tf.shape(image)
    height, width = shape[0], shape[1]

    def crop():
        offset_height = (height - target_height) // 2
        offset_width = (width - target_width) // 2
        return tf.image.crop_to_bounding_box(image, offset_height, offset_width, target_height, target_width)

    def resize():
        return tf.image.resize(image, [target_height, target_width])

    image = tf.cond(
        tf.logical_and(height >= target_height, width >= target_width),
        true_fn=crop,
        false_fn=resize
    )
    return image, label

def one_hot_encode(image, label):
    label = tf.one_hot(label, depth=classes_dim)
    return image, label

In [5]:
"""
Load data
"""
train = tf.keras.utils.image_dataset_from_directory(
    SPECTROGRAMS_DIR,
    validation_split=0.2,
    subset='training',
    image_size=(image_height, image_width),
    batch_size=batch_size,
    seed=123
)

valid = tf.keras.utils.image_dataset_from_directory(
    SPECTROGRAMS_DIR,
    validation_split=0.2,
    subset='validation',
    image_size=(image_height, image_width),
    batch_size=batch_size,
    seed=123
)

train = train.map(one_hot_encode)
valid = valid.map(one_hot_encode)

Found 41186 files belonging to 6 classes.
Using 32949 files for training.
Found 41186 files belonging to 6 classes.
Using 8237 files for validation.


In [6]:
"""
Train custom model
"""

model_input = keras.Input(shape=(image_height, image_width, 3))
x = model_input
x = layers.Resizing(image_height, image_width)(x)
x = layers.Rescaling(1.0 / 255)(x)

x = layers.Conv2D(16, (3, 3), activation="relu", padding="same", kernel_initializer="he_normal")(x)
x = layers.Conv2D(16, (3, 3), activation="relu", padding="same", kernel_initializer="he_normal")(x)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)

x = layers.Conv2D(32, (3, 3), activation="relu", padding="same", kernel_initializer="he_normal")(x)
x = layers.Conv2D(32, (3, 3), activation="relu", padding="same", kernel_initializer="he_normal")(x)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)

x = layers.Conv2D(64, (3, 3), activation="relu", padding="same", kernel_initializer="he_normal")(x)
x = layers.Conv2D(74, (3, 3), activation="relu", padding="same", kernel_initializer="he_normal")(x)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)

x = layers.Conv2D(128, (3, 3), activation="relu", padding="same", kernel_initializer="he_normal")(x)
x = layers.Conv2D(128, (3, 3), activation="relu", padding="same", kernel_initializer="he_normal")(x)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)

x = layers.Flatten()(x)
x = layers.Dense(64, activation="relu", kernel_initializer="he_normal")(x)
x = layers.Dropout(0.25)(x)
x = layers.Dense(64, activation="relu", kernel_initializer="he_normal")(x)
x = layers.Dropout(0.25)(x)

model_output = layers.Dense(classes_dim, activation="softmax")(x)
custom_model = keras.Model(inputs=model_input, outputs=model_output)
custom_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)
custom_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 resizing (Resizing)         (None, 128, 128, 3)       0         
                                                                 
 rescaling (Rescaling)       (None, 128, 128, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 128, 128, 16)      448       
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 16)      2320      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 64, 16)       0         
 )                                                               
                                                             

In [ ]:
"""
Train model
"""
history_custom = custom_model.fit(train, validation_data=valid, epochs=epochs_total)

plt.plot(history_custom.history["accuracy"], label="accuracy")
plt.plot(history_custom.history["val_accuracy"], label="val_accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

Epoch 1/60
1030/1030 [==============================] - 42s 30ms/step - loss: 1.6380 - accuracy: 0.2979 - val_loss: 1.4072 - val_accuracy: 0.4333
Epoch 2/60
1030/1030 [==============================] - 48s 47ms/step - loss: 1.3496 - accuracy: 0.4541 - val_loss: 1.1549 - val_accuracy: 0.5521
Epoch 3/60
1030/1030 [==============================] - 44s 42ms/step - loss: 1.1896 - accuracy: 0.5293 - val_loss: 1.0595 - val_accuracy: 0.5871
Epoch 4/60
1030/1030 [==============================] - 48s 46ms/step - loss: 1.0903 - accuracy: 0.5734 - val_loss: 1.0133 - val_accuracy: 0.6143
Epoch 5/60
1030/1030 [==============================] - 43s 42ms/step - loss: 1.0075 - accuracy: 0.6053 - val_loss: 0.8907 - val_accuracy: 0.6578
Epoch 6/60
1030/1030 [==============================] - 45s 44ms/step - loss: 0.9346 - accuracy: 0.6380 - val_loss: 0.8731 - val_accuracy: 0.6725
Epoch 7/60
1030/1030 [==============================] - 50s 49ms/step - loss: 0.8624 - accuracy: 0.6699 - val_loss: 0.8041 -

In [ ]:
"""
Save model
"""
custom_model.save(f"{TF_MODELS_DIR}/{model_name}")

In [ ]:
"""
Convert SavedModel (TensorFlow) to .tflite
"""
converter = tf.lite.TFLiteConverter.from_saved_model(f"{TF_MODELS_DIR}/{model_name}")
converter.experimental_new_converter = True
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS]
converter.allow_custom_ops = False
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

with open(f"{TFLITE_MODELS_DIR}/{model_name}.tflite", "wb") as f:
  f.write(tflite_model)